In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
import import_ipynb
#import Segmentation
import time
import preds

importing Jupyter notebook from preds.ipynb
importing Jupyter notebook from Captcha_seg.ipynb
1080


/home/bg/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


Train on 864 samples, validate on 216 samples
Epoch 1/20
864/864 [==============================] - 13s 14ms/step - loss: 3.5899 - acc: 0.0382 - val_loss: 3.5898 - val_acc: 0.0139
Epoch 2/20
864/864 [==============================] - 12s 13ms/step - loss: 3.5475 - acc: 0.0544 - val_loss: 3.4664 - val_acc: 0.0602
Epoch 3/20
864/864 [==============================] - 11s 13ms/step - loss: 2.9710 - acc: 0.1713 - val_loss: 2.7004 - val_acc: 0.2500
Epoch 4/20
864/864 [==============================] - 11s 13ms/step - loss: 2.1781 - acc: 0.3762 - val_loss: 2.1744 - val_acc: 0.3796
Epoch 5/20
864/864 [==============================] - 12s 13ms/step - loss: 1.5632 - acc: 0.5521 - val_loss: 1.9247 - val_acc: 0.4954
Epoch 6/20
864/864 [==============================] - 11s 13ms/step - loss: 1.0612 - acc: 0.6887 - val_loss: 1.7916 - val_acc: 0.5278
Epoch 7/20
864/864 [==============================] - 12s 13ms/step - loss: 0.7036 - acc: 0.7870 - val_loss: 1.7439 - val_acc: 0.5463
Epoch 8/20
864/8

In [2]:
CATEGORIES = set()
names =[]
for i in os.listdir('test/'):
    names.append(i.split('.')[0])
for i in names:
    for x in list(i):
        CATEGORIES.add(x)
CATEGORIES=list(CATEGORIES)
print(CATEGORIES)

['K', 'E', 'Q', 'F', '8', 'H', 'X', 'A', 'M', '3', 'W', 'J', 'B', '5', '6', 'N', '7', 'P', 'R', 'Y', 'S', '4', 'G', 'C']


In [3]:
#print(len(train_data))

In [4]:
import random

#random.shuffle(train_data)

In [5]:
#for sample in train_data[:10]:
#    print(sample[1])

In [6]:
X = []
y = []

for features, label in train_data:
    X.append(features)
    y.append(label)
    
X = np.array(X).reshape(-1, IMG_SIZE, IMG_SIZE, 1)

import pickle

pickle_out = open("X.pickle", "wb")
pickle.dump(X, pickle_out)
pickle_out.close()
pickle_out = open("y.pickle", "wb")
pickle.dump(y, pickle_out)
pickle_out.close()

pickle_in= open("X.pickle", "rb")
X = pickle.load(pickle_in)
X.shape[1:]

import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Dropout,Activation, Flatten, Conv2D, MaxPooling2D

X = pickle.load(open("X.pickle","rb"))
y = pickle.load(open("y.pickle","rb"))

X = X/255.0

model = Sequential()

model.add(Conv2D(64,(3,3),input_shape= X.shape[1:]))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(64,(3,3)))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(64,(3,3)))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation("relu"))


model.add(Dense(19))
model.add(Activation("softmax"))

from keras.utils import to_categorical
y_binary = to_categorical(y)

model.compile(loss="categorical_crossentropy",
             optimizer="adam",
             metrics=['accuracy'])

model.fit(X,y_binary,batch_size=128, epochs=20, validation_split = 0.2)


In [7]:
#loss_value, metrics_value = model.evaluate(X,y_binary,batch_size=10)

In [8]:
#print(loss_value, metrics_value)

In [9]:
import random

In [10]:
def similarity(a,b):
    i =0
    for x,y in zip(a,b):
        if x==y:
            i+= 1
    percentage = (i/5) * 100
    return percentage

def prepare(filepath):
    IMG_SIZE = 50
    img_array = cv2.imread(filepath, cv2.IMREAD_GRAYSCALE)
    new_array = cv2.resize(img_array, (IMG_SIZE,IMG_SIZE))
    #plt.imshow(new_array,'gray')
    return new_array.reshape(-1, IMG_SIZE,IMG_SIZE,1)

total_time = 0
score = 0
count = 0
done = []
list_ = preds.predictions('test',CATEGORIES)
CAPTCHA_DIR = list_[0]
for x in CAPTCHA_DIR:
    captchas = random.choice(CAPTCHA_DIR)
    if captchas in done:
        continue
    else:
        done.append(captchas)
        start_time = time.time()
        predict_list= []
        print('\n')
        print('Actual: ',captchas[0][0])

        files = []
        for segments in list_:
            files.append(segments)

        #for segments in files:
        #    filepath = (SEGMENTS_DIR,segments)
        #    prediction = model.predict([prepare(filepath)])
        #    predict_list.append(CATEGORIES[np.argmax(prediction[0])])

        elapsed_time = time.time()-start_time
        time.sleep(captchas[1])
        total_time += elapsed_time
        final_prediction = ''.join(predict_list)
        #if captcha_name[0]== final_prediction:
        #    score +=1
        score = list_[1]
        print('Predicted:',captchas[0][1],'\n')
        print('Elapsed time= ',captchas[1],'s')
        total_time +=captchas[1]
        print('<------------------------------->')    
    

print('Total number of CAPTCHAS broken: ',score)    
print('Total number of CAPTCHAS attempted: ',len(list_[0]))
print('Total time taken= ',total_time,'s')
print('Accuracy= ', (score/len(list_[0])*100))
#model = tf.keras.models.load_model('captcha_alpha.model')
#prediction = model.predict([prepare('3.png')])
#print(len(prediction[0]))
#print(prediction[0])
#print(CATEGORIES[np.argmax(prediction[0])])



Actual:  8YPH
Predicted: 8YPH 

Elapsed time=  0.8356056569083832 s
<------------------------------->


Actual:  GGF4
Predicted: MW88 

Elapsed time=  0.30935545056190017 s
<------------------------------->


Actual:  X5MJ
Predicted: BRMP 

Elapsed time=  0.6372613548976934 s
<------------------------------->


Actual:  48RK
Predicted: 48RK 

Elapsed time=  0.8025137912269099 s
<------------------------------->


Actual:  37N3
Predicted: 37N3 

Elapsed time=  0.8487233266336904 s
<------------------------------->


Actual:  QHH3
Predicted: QHH3 

Elapsed time=  0.21940598910187503 s
<------------------------------->


Actual:  44K3
Predicted: 3WR8 

Elapsed time=  0.01384854070673791 s
<------------------------------->


Actual:  4C73
Predicted: WWMK 

Elapsed time=  0.6602420934946651 s
<------------------------------->


Actual:  E3NH
Predicted: AGWX 

Elapsed time=  0.01394631802302726 s
<------------------------------->


Actual:  68J7
Predicted: 68J7 

Elapsed time=  0.886862319

Predicted: PFE8 

Elapsed time=  0.5489312065231646 s
<------------------------------->


Actual:  5XC5
Predicted: 5XC5 

Elapsed time=  0.09786884038039512 s
<------------------------------->


Actual:  3CJA
Predicted: 3CJA 

Elapsed time=  0.990818987680766 s
<------------------------------->


Actual:  GM7N
Predicted: GM7N 

Elapsed time=  0.768027510869533 s
<------------------------------->


Actual:  5HGW
Predicted: BCMY 

Elapsed time=  0.5896942173640908 s
<------------------------------->


Actual:  RPME
Predicted: 73MJ 

Elapsed time=  0.6223123844122614 s
<------------------------------->


Actual:  X7KE
Predicted: X7KE 

Elapsed time=  0.07038771952848766 s
<------------------------------->


Actual:  PS5K
Predicted: HCPQ 

Elapsed time=  0.6954516854621487 s
<------------------------------->


Actual:  KGF8
Predicted: KGF8 

Elapsed time=  0.36286335752251364 s
<------------------------------->


Actual:  W58A
Predicted: NNQQ 

Elapsed time=  0.9356605245982579 s
<--------

Predicted: RY5B 

Elapsed time=  0.8901248169981344 s
<------------------------------->


Actual:  6BHG
Predicted: 6BHG 

Elapsed time=  0.8672658044641193 s
<------------------------------->


Actual:  JK7M
Predicted: 3B75 

Elapsed time=  0.5045508226991412 s
<------------------------------->


Actual:  MQRJ
Predicted: AW7P 

Elapsed time=  0.6535079214630557 s
<------------------------------->


Actual:  QCWE
Predicted: 654M 

Elapsed time=  0.8813438706728584 s
<------------------------------->


Actual:  76CE
Predicted: 76CE 

Elapsed time=  0.6812396745169068 s
<------------------------------->


Actual:  8AFM
Predicted: 8AFM 

Elapsed time=  0.6632766851002192 s
<------------------------------->


Actual:  74AQ
Predicted: NS3M 

Elapsed time=  0.3598710354363478 s
<------------------------------->


Actual:  3JY8
Predicted: 3JY8 

Elapsed time=  0.7508827665702555 s
<------------------------------->


Actual:  33ER
Predicted: Q4FF 

Elapsed time=  0.12090317652385796 s
<--------

Predicted: 8HJC 

Elapsed time=  0.34230392793387765 s
<------------------------------->


Actual:  XWFF
Predicted: GXXR 

Elapsed time=  0.6789320803939899 s
<------------------------------->


Actual:  6F6Y
Predicted: 57H8 

Elapsed time=  0.39244502868811415 s
<------------------------------->


Actual:  FHYC
Predicted: FHYC 

Elapsed time=  0.2960201734581689 s
<------------------------------->


Actual:  KNSW
Predicted: KNSW 

Elapsed time=  0.44947578486602113 s
<------------------------------->


Actual:  PNPX
Predicted: PNPX 

Elapsed time=  0.6157525261167576 s
<------------------------------->


Actual:  S74X
Predicted: 5P6Y 

Elapsed time=  0.8312490823341527 s
<------------------------------->


Actual:  GM7N
Predicted: GM7N 

Elapsed time=  0.8749953269457099 s
<------------------------------->


Actual:  8Q6P
Predicted: 8Q6P 

Elapsed time=  0.7010001212525984 s
<------------------------------->


Actual:  XF5F
Predicted: XF5F 

Elapsed time=  0.6299504555295808 s
<------